# Creating a Pydantic Model

To define a Pydantic model, we have to inherit from the `BaseModel` class.

In [24]:
from pydantic import BaseModel


class Person(BaseModel):
    first_name: str
    last_name: str
    age: int


p = Person(first_name="Ayda", last_name="Galois", age=20)

# We can access using
print(str(p))

print(repr(p))

print(p.model_fields)

first_name='Ayda' last_name='Galois' age=20
Person(first_name='Ayda', last_name='Galois', age=20)
{'first_name': FieldInfo(annotation=str, required=True), 'last_name': FieldInfo(annotation=str, required=True), 'age': FieldInfo(annotation=int, required=True)}


You'll notice that each field is listed, and one interesting thing is that `required=True` attached to each field. And indeed, if we were to try and create a model instance without passing in a value for any of those fields, we would get an exception. And indeed, if we were to try and create a model instance without passing in a value for any of those fields, we would get an exception. Pydantic raise the `ValidationError` whenever something goes wrong trying to create a model instance.

In [11]:
from pydantic import ValidationError

Now let's see what happens when we don't provide a required field:

In [14]:
try:
    Person(last_name='Galois')
except ValidationError as ex:
    print(ex)

2 validation errors for Person
first_name
  Field required [type=missing, input_value={'last_name': 'Galois'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
age
  Field required [type=missing, input_value={'last_name': 'Galois'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


As you can see, Pydantic reports back on **all** the validation errors it encounters.

By default, Pydantic does not stop validating data when it encounters the first validation issue. Instead it continues validating everything, and then returns a complete list of all the validation errors.

This can be very useful for REST APIs, where you typically want to let an API caller know **everything** that was wrong with their JSON payload.

I mentioned that Pydantic classes are just regular Python classes. They obtain their special functionality by inheriting from `BaseModel`, but they are standard classes. Which means we can add properties and methods to it.

In [15]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

    @property
    def display_name(self):
        return f"{self.first_name} {self.last_name[0]}"

In [16]:
p = Person(first_name="Evariste", last_name="Galois", age=20)
p

Person(first_name='Evariste', last_name='Galois', age=20)

In [17]:
p.display_name

'Evariste G'

Now, we have to be a bit careful here. Pydantic will perform validation when it loads data (**deserializes** data) to create a model instance.

In particular, we have type hints attached to each field:

In [25]:
try:
    Person(first_name='Evariste', last_name='Galois', age='twenty')
except ValidationError as ex:
    print(ex)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='twenty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing


As you can see, our incorrect input for `age` is trapped by Pydantic.

However, by default, this does not happen when we set a field value once the model instance has been created:

In [21]:
p.age = "twenty"

In [22]:
p

Person(first_name='Evariste', last_name='Galois', age='twenty')

In [29]:
from pathlib import Path as p
from copy import deepcopy
import re

REGX_PATTERN = "( - | )"
REPLACE_WITH = "-"
# Get current directory
rel_path = p.cwd()
# Get all files
files = [file for file in rel_path.iterdir()]
# Get all filenames
names = [file.name for file in files]
# Do type casting for each name
fmt_names = [re.sub(REGX_PATTERN, REPLACE_WITH, name) for name in names]
# Change each file name
for file, name in zip(files, fmt_names):
    file.with_name(name)

See? The change went through.

In a later video, I'll show you how we can configure our Pydantic model so that doing this will also raise a `ValidationError` exception.